## Install & Import Libraries

In [1]:
pip install tqdm

In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re
from nltk import sent_tokenize
from nltk import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## PreProcess Data

In [3]:
!wget -O dataset.csv "https://drive.google.com/uc?export=download&id=1s128Fg1-0udJzZb3--3lWY5brFacCQyT&confirm=NEW_FILE_CONFIRM_CODE"

--2024-02-03 18:37:24--  https://drive.google.com/uc?export=download&id=1s128Fg1-0udJzZb3--3lWY5brFacCQyT&confirm=NEW_FILE_CONFIRM_CODE
Resolving drive.google.com (drive.google.com)... 142.250.1.100, 142.250.1.102, 142.250.1.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.1.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1s128Fg1-0udJzZb3--3lWY5brFacCQyT&export=download [following]
--2024-02-03 18:37:25--  https://drive.usercontent.google.com/download?id=1s128Fg1-0udJzZb3--3lWY5brFacCQyT&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 209.85.145.132, 2607:f8b0:4001:c1f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|209.85.145.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66212309 (63M) [application/octet-stream]
Saving to: ‘dataset.csv’

dataset.csv         100%[================

In [49]:
df = pd.read_csv('dataset.csv')
df = df[:5000]
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
df['review'][700]

"Unfortunately the only spoiler in this review is that there's nothing to spoil about that movie.Even if B. Mattei had never done any master piece he use to do his job with a bit of humor and craziness that made him a fun Eurotrash director. But for the last 10 years he seemed to have lost it.This film is just empty, nothing at all to wake us up from the deep sleep you sink into after the first 10 min.No sex, no blood(it's suppose to be about snuff?),no actors, no dialogs, just as bad as an 90'T.V film.It's even worse than his last cannibals and zombies epics.So Rest in peace Bruno, you will stay in our minds forever anyway, thanks to such unforgettable gems as:Zombi 3, Robowar,Rats, l'altro inferno,Virus, Cruel jaws and few others.So except if you want to see B Mattei possessed by jess Franco's spirit's new film, pass on this one.But if you don't know this nice artisan's career track down his old films and have fun."

In [10]:
corpus=[]
corpus_words=[]
for item in tqdm(df['review']):
  sentences = sent_tokenize(item)
  temp2 = []
  for sent in sentences:
    sent = re.sub(r'<br',' ',sent)
    sent = sent.lower()
    words = word_tokenize(sent)
    punctuation = string.punctuation
    stop_words = set(stopwords.words('english'))
    temp = [word for word in words if (word[0] not in punctuation) and (word not in stop_words)] #Remove punctuation
    lemmatizer = WordNetLemmatizer()
    preprocessed_words = [lemmatizer.lemmatize(word) for word in temp]
    corpus_words.append(preprocessed_words)
    temp2.append(' '.join(preprocessed_words))
  temp2 = '. '.join(temp2)
  corpus.append(temp2)
corpus[:2]

100%|██████████| 5000/5000 [00:28<00:00, 173.04it/s]


["one reviewer mentioned watching 1 oz episode hooked. right exactly happened first thing struck oz brutality unflinching scene violence set right word go. trust show faint hearted timid. show pull punch regard drug sex violence. hardcore classic use word called oz nickname given oswald maximum security state penitentary. focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda. em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away would say main appeal show due fact go show would n't dare. forget pretty picture painted mainstream audience forget charm forget romance oz n't mess around. first episode ever saw struck nasty surreal could n't say ready watched developed taste oz got accustomed high level graphic violence. violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street s

## BOW

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X_BOW = cv.fit_transform(corpus).toarray()

In [14]:
y = pd.get_dummies(df['sentiment'],drop_first=True)

## TF-IDF

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500)
X_TF_IDF = tv.fit_transform(corpus).toarray()

## Word2Vec

In [16]:
from gensim.models import Word2Vec
model = Word2Vec(corpus_words, window=5, vector_size=100, min_count = 2)

In [17]:
model.wv.index_to_key[0:10]

['movie',
 'film',
 "n't",
 'one',
 'like',
 'time',
 'good',
 'character',
 'would',
 'get']

In [69]:
def avg_word2vec(sentence):
  words = word_tokenize(sentence)
  if all(word not in model.wv.index_to_key for word in words):
    return None
  return np.mean([model.wv[word] for word in words if word in model.wv.index_to_key],axis=0)

In [70]:
X_word2vec = []
for item in tqdm(df['review']):
  ans = avg_word2vec(item)
  if ans is None:
    ans = np.array([0]*100, dtype="float32")
  X_word2vec.append(ans)

100%|██████████| 5000/5000 [08:44<00:00,  9.54it/s]


## Divide data

In [74]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split

In [75]:
X_BOW_train, X_BOW_test, y_train_bow, y_test_bow = train_test_split(X_BOW, y, test_size=20, random_state=2)
X_TF_IDF_train, X_TF_IDF_test, y_train_tfidf, y_test_tfidf = train_test_split(X_TF_IDF, y, test_size=20, random_state=2)
X_w2v_train, X_w2v_test, y_train_w2v, y_test_w2v = train_test_split(X_word2vec, y, test_size=20, random_state=2)

## Train models for Naive Bayes

In [76]:
sentiment_detection_model_BOW = MultinomialNB().fit(X_BOW_train,y_train_bow)
sentiment_detection_model_TF_IDF = MultinomialNB().fit(X_TF_IDF_train,y_train_tfidf)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Evalution of Naive Bayes

In [77]:
y_pred_BOW = sentiment_detection_model_BOW.predict(X_BOW_test)
y_pred_tfidf = sentiment_detection_model_TF_IDF.predict(X_TF_IDF_test)

In [78]:
print(accuracy_score(y_test_bow,y_pred_BOW))
print(accuracy_score(y_test_tfidf,y_pred_tfidf))

0.75
0.8


In [79]:
print(classification_report(y_pred_BOW,y_test_bow))
print(classification_report(y_pred_tfidf,y_test_tfidf))

              precision    recall  f1-score   support

           0       0.67      0.89      0.76         9
           1       0.88      0.64      0.74        11

    accuracy                           0.75        20
   macro avg       0.77      0.76      0.75        20
weighted avg       0.78      0.75      0.75        20

              precision    recall  f1-score   support

           0       0.75      0.90      0.82        10
           1       0.88      0.70      0.78        10

    accuracy                           0.80        20
   macro avg       0.81      0.80      0.80        20
weighted avg       0.81      0.80      0.80        20



## Train models using RandomForest

In [80]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier())

In [81]:
sentiment_detection_model_w2v = forest.fit(X_w2v_train,y_train_w2v)

<ipython-input-81-aa13494d912e>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  sentiment_detection_model_w2v = forest.fit(X_w2v_train,y_train_w2v)


In [86]:
sentiment_detection_model_BOW = forest.fit(X_BOW_train,y_train_bow)
sentiment_detection_model_TF_IDF = forest.fit(X_TF_IDF_train,y_train_tfidf)


<ipython-input-86-ae54bf1e0025>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  sentiment_detection_model_BOW = forest.fit(X_BOW_train,y_train_bow)
<ipython-input-86-ae54bf1e0025>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  sentiment_detection_model_TF_IDF = forest.fit(X_TF_IDF_train,y_train_tfidf)


0.75
0.8


### Evalution of RandomForest

In [82]:
y_pred_w2v = sentiment_detection_model_w2v.predict(X_w2v_test)
y_pred_BOW = sentiment_detection_model_BOW.predict(X_BOW_test)
y_pred_tfidf = sentiment_detection_model_TF_IDF.predict(X_TF_IDF_test)

In [88]:
print(accuracy_score(y_test_bow,y_pred_BOW))
print(accuracy_score(y_test_tfidf,y_pred_tfidf))
print(accuracy_score(y_test_w2v,y_pred_w2v))

0.75
0.8
0.55


In [84]:

print(classification_report(y_pred_w2v,y_test_w2v))

              precision    recall  f1-score   support

           0       0.50      0.67      0.57         9
           1       0.62      0.45      0.53        11

    accuracy                           0.55        20
   macro avg       0.56      0.56      0.55        20
weighted avg       0.57      0.55      0.55        20



In [87]:
print(classification_report(y_pred_BOW,y_test_bow))
print(classification_report(y_pred_tfidf,y_test_tfidf))

              precision    recall  f1-score   support

           0       0.92      0.73      0.81        15
           1       0.50      0.80      0.62         5

    accuracy                           0.75        20
   macro avg       0.71      0.77      0.72        20
weighted avg       0.81      0.75      0.76        20

              precision    recall  f1-score   support

           0       0.92      0.79      0.85        14
           1       0.62      0.83      0.71         6

    accuracy                           0.80        20
   macro avg       0.77      0.81      0.78        20
weighted avg       0.83      0.80      0.81        20

